# **<font color='crimson'>Оценка результатов эксперимента на этапе планирования A/B-теста</font>**

---

**Выполнил**: Юмаев Егор

---

На этапе планирования A/B-теста с помощью доступных статистических онлайн-калькуляторов имеется возможность выставить желаемые значения метрик и получить соответствующее им значение размера выборки.

С целью повышения надежности A/B-теста посредством применения случайных чисел, метода Монте-Карло целесообразно провести дополнительную проверку, подтверждающую, что уровень выборки достаточен для достижения запланированных метрик A/B-теста.

## <font color='green'>**1 Случайность в A/B-тестах**</font>

---

In [1]:
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest
from tqdm.notebook import tqdm
from tqdm import tqdm

Для симуляции случайности в A/B-тестах воспользуемся модулем **random** библиотеки **numpy**, в котором есть функции, генерирующие случайные числа. Для симуляции исходов **0** и **1** (моделирование **конверсии** - доли покупателей, совершивших покупку) воспользуемся функцией **binomial**.

In [2]:
# в функции binomial вероятность выпадения 1 (единицы)
# зададим, равной 3%
np.random.binomial(1, 0.03, size = 10)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

В случае небольшой выборки при конверсии 3% преимущественно будем получать результаты, сильно отличающиеся от 3%. В основном запуски функции binomial для size = 10 будут показывать нулевую конверсию. Но если среди результатов окажется хотя бы одна конверсия, результат сразу составит целых 10%, что значимо отличается от 3%.



In [3]:
# результат для случая, когда при вероятности выпадения
# единицы в 3% получены две конверсии (две единицы)
np.mean([0, 0, 0, 0, 1, 0, 0, 0, 0, 1])

0.2

При небольших размерах выборки большую роль играет **случайность**.

**Увеличим** размер выборки до 100 значений.

In [4]:
# вместо вывода фактических результатов
# сразу выведем результат конверсии по отдельному запуску
np.random.binomial(1, 0.03, size = 100).mean()

0.02

Даже если истинная конверсия 3% и мы точно это знаем, то по причине маленькой выборки нет стабильности результатов: конверсия в зависимости от запуска колеблется от 0 и выше, преимущественно в диапазоне 0-6%.

**Увеличим** размер выборки до 1000 значений.

In [5]:
np.random.binomial(1, 0.03, size = 1_000).mean()

0.034

**Увеличим** размер выборки до 10000 значений.

In [6]:
np.random.binomial(1, 0.03, size = 10_000).mean()

0.0313

Фактическая конверсия, получаемая на выборке с size = 1000 уже более-менее близка к истинной конверсии. Так провляет себя популярный в статистике **закон больших чисел** - при увеличении размера выборки выборочное среднее приближается к истинному размеру выборки.

Таким образом, **случайность** преимущественно проявляется тогда, когда мы располагаем маленькой выборкой: чем меньше выборка, тем большей случайности мы подвержены.

Продолжим анализ, применив симуляцию для двух групп (одна из них будет тестовой, другая контрольной) с помощью случайных чисел и серии экспериментов. Этот подход называется **методом Монте-Карло**.

Предположим, что в группе **а** истинная конверсия равна **3%**, а в группе **b** - **5%**.

In [7]:
# проверим результат разового запуска эксперимента
np.random.seed(1)

a = np.random.binomial(1, 0.03, size = 1_000).mean()
b = np.random.binomial(1, 0.05, size = 1_000).mean()
print(a, b)

0.032 0.054


Размер выборки size = 1000 больше, чем 10 или 100. Но и эта выборка не является значительной, а, значит, не может обеспечить необходимую стабильность.

С помощью **метода Монте-Карло** ответим на вопрос: 'Может ли сложиться такая ситуация, что фактическая конверсия в первой группе **a** будет **выше**, чем фактическая конверсия во второй группе **b**'?

Таким образом мы моделируем эксперимент, заранее зная, что конверсия в группе **a**  имеет 3%, конверсия в группе **b** имеет 5%. Однако, собранные данные могут дать нам неверное представление из-за **случайности** по причине недостаточного размера выборки.

Сгенерируем ситуацию расчета конверсии 1000 раз.

In [8]:
# моделируем расчет конверсии, сгенерировав результаты 1000 раз
n = 1000
result = []

for _ in range(n):
    a = np.random.binomial(1, 0.03, size = 1_000).mean()
    b = np.random.binomial(1, 0.05, size = 1_000).mean()
    result.append([a, b])

In [9]:
# выведем первые пять пар результата
result[:5]

[[0.021, 0.054], [0.033, 0.05], [0.026, 0.051], [0.035, 0.04], [0.025, 0.053]]

In [10]:
# сохраним результаты эксперимента в датсет
df = pd.DataFrame(result, columns = ['a', 'b'])

# выведем первые пять строк сформированного датасет
df.head()

,a,b
0,0.021,0.054
1,0.033,0.050
2,0.026,0.051
3,0.035,0.040
4,0.025,0.053


Получив результаты эксперимента, ответим на вопрос: может ли сложиться так, что конверсия в первой группе окажется больше конверсии во второй группе. При условии, что в реальности конверсия во творой группе больше, чем в первой.

In [11]:
# отфильтруем строки датасета df, в которых
# конверсия в группе а больше, чем в группе b
df[df['a'] > df['b']]

,a,b
31,0.041,0.039
136,0.026,0.024
183,0.039,0.038
199,0.032,0.028
216,0.041,0.031
303,0.039,0.037
433,0.038,0.037
488,0.033,0.028
603,0.031,0.029
741,0.036,0.032


Действительно, ситуации, когда полученная в результате эксперимента конверсия в первой группе оказалась больше конверсии во второй группе, имеются. Таким образом, в ситуации, когда мы заранее знаем, что конверсия в первой группе меньше, чем конверсия во второй, из-за **случайности**, обусловленной малым размером выборок, мы можем получить обратный результат, который противоречит реальному положению дел.

Принимать решение в условиях случайности помогает **статистический тест**. Метрики статистического теста:

* **True Positive Rate** (**Sensitivity**)

* **False Positive Rate** (**1 - Specificity**)

* **MDE**, -

позволяют оценить уверенность с которой мы формулируем решение. Т.е. указанные метрики позволяют оценить надежность теста. Но посчитать метрики для оценки конверсии аналогично оценке качества, например, тестов на беременность с бинарным ответом, затруднительно. Поэтому получить метрики статистического теста можно двумя способами:

(1) с помощью **математических методов**, позволяющих посчитать метрики **заранее**, - представить, какие метрики мы получим в той или иной ситуации;

(2) с помощью **метода Монте-Карло**.

## <font color='green'>**2 Оценка результатов A/B-теста с помощью метода Монте-Карло и онлайн-калькулятора Глеба Михайлова на этапе планирования эксперимента**</font>

---

Предварительно в **онлайн-калькуляторе Глеба Михайлова** рассчитали необходимый размер группы 1484 при следующих параметрах:

* Baseline Conversion Rate 0.03

* Minimum Detectable Effect 0.02

* Test to Control Group Ratio 1.0

* Power (Desired minimum True Positive Rate) 0.8

* Significance (Desired maximum False Positive Rate) 0.05

Ссылка на **онлайн-калькулятор Глеба Михайлова**:

https://glebmikha.github.io/ab-test-calculator-by-gleb-mikhaylov/

In [12]:
# вычислим средний размер конверсии в контрольной и тестовой группах

np.random.seed(7)

a = np.random.binomial(1, 0.03, size = 1_484).mean()
b = np.random.binomial(1, 0.05, size = 1_484).mean()

print(a, b)

0.02830188679245283 0.04784366576819407


Предварительная оценка с помощью виртуального эксперимента показывает, что статистический тест может уловить разницу в конверсиях в 0.02 при размере выборки в 1484. При этом, планируя эксперимент, мы заранее знаем, что конверсии в двух группах различаются.

В аргументах функции **np.random.binomial** уберем размер выборки size, а первый аргумент заменим на размер выборки, вероятность успеха сохраним. В результате функция выдаст количество **'успехов'** при заданной вероятности.

In [13]:
# получим число конверсий
# в контрольной и тестовой группах

np.random.seed(7)

a = np.random.binomial(1_484, 0.03)
b = np.random.binomial(1_484, 0.05)

print(a, b)

38 76


Подставив рассчитанное число конверсий в контрольной и тестовой группах и размер выборки в **онлайн-калькулятор Глеба Михайлова** получаем ответ о том, что необходимо отвергнуть нулевую гипотезу, конверсии выборок различаются.

Однако, без фиксации **np.random.seed()** в один из экспериментальных запусков было получено число конверсий для группы **а** 49, для группы **b** 52. Результат, полученный с помощью **онлайн-калькулятора Глеба Михайлова**: принимаем нулевую гипотезу, разницы в конверсии между двумя группами нет. Хотя, планируя эксперимент, мы точно знаем, что разница в конверсии в двух группах есть.

Это **False Negative** - мы пропустили эффект, когда в действительности он есть. Вероятность такой ошибки заложена в калькулятор и составляет 5% (**уровень значимости**).

Чтобы **на этапе планирования эксперимента** убедиться, что при заданных параметрах и размере выборки статистический тест сможет уловить разницу, если она есть, требуется провести большое количество экспериментов. Это позволит убедиться, действительно ли в реультате применения теста получим желаемую мощность и уровень значимости.

То есть, по итогам многократной симуляции (моделирования) экспериментов, необходимо установить, в каком количестве случаев (**мощность** / **Power** / **True Positive Rate**) статистический тест сможет уловить разницу, когда она действительно есть.


In [14]:
# функция, позволяющая смоделировать многократный запуск эксперимента
# и рассчитать мощность при заданных параметрах;
# conv_a и conv_b - конверсии в группах a и b

def test(conv_a, conv_b, size_a, size_b, significance=0.05):
    _, p_value = proportions_ztest(
        [conv_a, conv_b],
        [size_a, size_b],
        alternative='two-sided')
    return p_value < significance

При конверсиях в группах a и b **онлайн-калькулятор Глеба Михайлова** дал отрицательный ответ: не отвергаем нулевую гипотезу о равенстве конверсий в группах, хотя в реальности конверси ив группах различаются.

Проверим, какй результат будет получен в результате применения **proportion_ztest**.

In [15]:
# применим функцию test к размерам конверсий
# в группах a и b в 49 и 52
test(49, 52, 1484, 1484)

False

**False** означает, что ответ отрицательный, различий в конверсиях двух групп нет (что не соответствует реальному положению дел).

Протестируем функцию test на значениях конверсии, которые точно должны показать наличие развитий: 44 и 78 соответственно для группы **a** и группы **b**.

In [16]:
# применим функцию test к размерам конверсий
# в группах a и b в 44 и 78
test(44, 78, 1484, 1484)

True

**True** означает, что результат положительный: отвергаем нулевую гипотезу о равенстве значений конверсий, конверсии двух групп различаются.

На следующих шагах проведем множество виртуальных экспериментов и проверим, получаем ли мы желаемые метрики статистического теста, или нет.

### <font color='green'>**2.1 Оценка True Positive Rate (Sensitivity)**</font>

---

Оценку **True Positive Rate (Sensitivity)** проведем с помощью генерации числа конверсий в группах 'a' и 'b', заранее зная, что разница между велиичнами конверсий в группах есть:

In [17]:
a = np.random.binomial(1_484, 0.03)
b = np.random.binomial(1_484, 0.05)

**True Positive Rate** можно зафиксировать только на реальном **positive**:

\begin{align}
\mathbf{TPR (Sensitivity)} = \frac{\mathbf{TP}}{\mathbf{Positive_{real}}}
\end{align}

Сгенерируем 1000 экспериментов для набора конверсий 49 (для группы 'a') и 52 (для группы 'b') и для каждого эксперимента посчитаем ответ True или False. После чего посчитаем число ответов  True и поделим на общее количество ответов.

In [18]:
# запускаем эксперимент 1000 раз
# для каждого запуска получим пару конверсий для тестовой и контрольной группы
n = 1000
result = []

for _ in range(n):
    a = np.random.binomial(1_484, 0.03)
    b = np.random.binomial(1_484, 0.05)
    result.append((a,b))

In [19]:
# выведем первые пять значений результата
result[:5]

[(52, 69), (45, 73), (64, 73), (48, 72), (52, 65)]

In [20]:
# сохраним результаты в таблицу
df = pd.DataFrame(result, columns = ['a', 'b'])

In [21]:
# выведем первые пять строк таблицы result
df.head()

,a,b
0,52,69
1,45,73
2,64,73
3,48,72
4,52,65


Каждая строка датасета df - результат отдельного виртуального эксперимента.

Посчитаем результат каждого виртуального теста, используя функцию **test**:

* True - решение положительное: считаем, что разница в конверсиях между двумя группами есть;

* False - решение отрицательное: считаем, что разницы в конверсиях между двумя группами нет.

In [22]:
# результат каждого виртуального теста True / False
# сохраним в отдельный столбец датасета df
df['test'] = df.apply(lambda row: test(row['a'], row['b'], 1484, 1484), axis=1)

In [23]:
# выведем последние пять строк датасета df
df.tail()

,a,b,test
995,40,68,True
996,46,80,True
997,59,86,True
998,53,80,True
999,49,66,False


Сходу можно увидеть, что среди результатов есть False, хотя мы знаем заранее, что таких результатов быть не должно - конверсии в двух группах точно различаются.

Поскольку при расчете начальных параметров мы заложили желаемую мощность теста (**True Positive Rate**) в 80%, результат **False** мы должны увидеть в 20% случаев, а **True** должно быть в 80% случаев.

In [24]:
# посчитаем процент результатов True с помощью среднего
df['test'].mean()

0.813

Желаемая мощность **Power (Desired minimum TPR)** должна быть не менее 80%, поэтому результат немного выше нас вполне устраивает.

Добавим, что у **метода Монте-Карло** есть ограничение по точности, которое определяется количеством проводимых виртуальных экспериментов. Если нам требуется более точная оценка для **TPR** теста, то нам следует повысить количество наблюдений. Например, можно провести не 1_000, а 10_000 виртуальных экспериментов.

Чем меньше MDE теста, тем больше повторений при проведени вирутальных экспериментов **методом Монте-Карло** требуется.

**В результате** применения случайных чисел, виртуальных экспериментов, **метода Монте-Карло** удалось подтвердить, что тест, который был запланирован, действительно обладает таким **Power (Desired minimum TPR)**, который в него был заложен. Была проведена 1000 экспериментов, причем заранее было известно, что разница есть, и в 80% случаев статистический тест эту разницу обнаружил, в 20% случаев - **не** обнаружил. Это точно соответствует тому, что было заложено в тест при планировании эксперимента.

### <font color='green'>**2.2 Оценка False Positive Rate (Significance)**</font>

---

Для оценки уровня значимости (**False Positive Rate**) в цикле, в котором формируются случайные величины конверсий в тестовой и контрольной группах, укажем равные конверсии. Теперь на этапе планирования эксперимента предполагаем, что разницы в уровне конверсий между двумя группами **нет**.

Также проведем не 1_000, а 100_000 повторений, - для повышения точности результатов применения **метода Монте-Карло**.

In [25]:
# запускаем эксперимент 100_000 раз
# для каждого запуска получим пару конверсий для тестовой и контрольной группы
n = 100_000
result = []

for _ in tqdm(range(n)):
    a = np.random.binomial(1_484, 0.03)
    b = np.random.binomial(1_484, 0.03)
    result.append((a,b))

100%|██████████| 100000/100000 [00:00<00:00, 183766.70it/s]


In [26]:
# выведем первые пять значений результата
result[:5]

[(56, 42), (44, 38), (41, 45), (47, 39), (43, 40)]

In [27]:
# сохраним результаты в таблицу
df = pd.DataFrame(result, columns = ['a', 'b'])

In [28]:
# выведем первые пять строк таблицы result
df.head()

,a,b
0,56,42
1,44,38
2,41,45
3,47,39
4,43,40


Каждая строка датасета df - результат отдельного виртуального эксперимента.

Проведено 100_000 виртуальных экспериментов, в которых мы заранее знаем, что никакой разницы между конверсиями в тестовой и контрольной группах нет.

Посчитаем результат каждого виртуального теста, используя функцию **test**:

* True - решение положительное: считаем, что разница в конверсиях между двумя группами есть;

* False - решение отрицательное: считаем, что разницы в конверсиях между двумя группами нет.

In [29]:
# результат каждого виртуального теста True / False
# сохраним в отдельный столбец датасета df
tqdm.pandas()

df['test'] = df.progress_apply(
    lambda row: test(row['a'], row['b'], 1484, 1484), axis=1)

100%|██████████| 100000/100000 [00:28<00:00, 3526.30it/s]


Тесты рассчитались: для каждого результата получен ответ. Посмотрим на результаты.

In [30]:
# выведем последние пять строк датасета df
df.tail()

,a,b,test
99995,39,54,False
99996,49,49,False
99997,37,42,False
99998,50,54,False
99999,28,50,True


В качестве результата ожидаются только **False**, однако же среди результатов виртуальных экспериментов встречаются и **True**, которых здесь быть не должно, т.к. мы точно знаем, что разницы в конверсиях между двумя группами нет.

Однако, статистический тест не идеален, и в 5% случаев, когда разницы нет, тест допускает ложное срабатывание, - он дает ответ, что разница есть.

In [31]:
# посчитаем процент результатов True с помощью среднего
df['test'].mean()

0.05092

По итогам проведения серии из 100 тысяч экспериментов и усреднения полученных результатов, оценка уровня значимости (**False Positive Rate**) оказалась равна 5%, как и было заложено при планировании теста.

### <font color='green'>**2.3 Оценка Minimal Detectable Effect (MDE)**</font>

---

Вернемся к ситуации, когда разница в конверсии между группами есть. И реальная разница соответствует **Minimal Detectable Effect**.

Запустим проверку и посмотрим, в каком количестве случаев тест обнаруживает разницу.

In [32]:
# запускаем эксперимент 100_000 раз
# для каждого запуска получим пару конверсий для тестовой и контрольной группы
n = 100_000
result = []

for _ in tqdm(range(n)):
    a = np.random.binomial(1_484, 0.03)
    b = np.random.binomial(1_484, 0.05)
    result.append((a,b))

100%|██████████| 100000/100000 [00:00<00:00, 388413.63it/s]


In [33]:
# выведем первые пять значений результата
result[:5]

[(39, 78), (39, 80), (41, 78), (48, 75), (36, 66)]

In [34]:
# сохраним результаты в таблицу
df = pd.DataFrame(result, columns = ['a', 'b'])

In [35]:
# выведем первые пять строк таблицы result
df.head()

,a,b
0,39,78
1,39,80
2,41,78
3,48,75
4,36,66


Посчитаем результат каждого виртуального теста, используя функцию **test**:

* True - решение положительное: считаем, что разница в конверсиях между двумя группами есть;

* False - решение отрицательное: считаем, что разницы в конверсиях между двумя группами нет.

In [36]:
# результат каждого виртуального теста True / False
# сохраним в отдельный столбец датасета df
tqdm.pandas()

df['test'] = df.progress_apply(
    lambda row: test(row['a'], row['b'], 1484, 1484), axis=1)

100%|██████████| 100000/100000 [00:19<00:00, 5189.23it/s]


Тесты рассчитались: для каждого результата получен ответ. Посмотрим на результаты.

In [37]:
# выведем последние пять строк датасета df
df.tail()

,a,b,test
99995,50,77,True
99996,46,77,True
99997,44,68,True
99998,45,84,True
99999,47,60,False


In [38]:
# посчитаем процент результатов True с помощью среднего
df['test'].mean()

0.79808

Видим, что тест обнаруживает **Minimal Detectable Effect (MDE)** почти в 80% случаев.

Проверим, что произойдет, если MDE понизить с 0.05 до 0.04. То есть предположим, что конверсия в контрольной группе 0.03, а в тестовой не 0.05, а 0.04.

In [39]:
# запускаем эксперимент 100_000 раз
# для каждого запуска получим пару конверсий для тестовой и контрольной группы
n = 100_000
result = []

for _ in tqdm(range(n)):
    a = np.random.binomial(1_484, 0.03)
    b = np.random.binomial(1_484, 0.04)
    result.append((a,b))

100%|██████████| 100000/100000 [00:00<00:00, 371531.45it/s]


In [40]:
# сохраним результаты в таблицу
df = pd.DataFrame(result, columns = ['a', 'b'])

In [41]:
# выведем первые пять строк таблицы result
df.head()

,a,b
0,62,66
1,28,55
2,49,54
3,43,65
4,49,83


Посчитаем результат каждого виртуального теста, используя функцию **test**:

* True - решение положительное: считаем, что разница в конверсиях между двумя группами есть;

* False - решение отрицательное: считаем, что разницы в конверсиях между двумя группами нет.

In [42]:
# результат каждого виртуального теста True / False
# сохраним в отдельный столбец датасета df
tqdm.pandas()

df['test'] = df.progress_apply(
    lambda row: test(row['a'], row['b'], 1484, 1484), axis=1)

100%|██████████| 100000/100000 [00:17<00:00, 5588.75it/s]


Тесты рассчитались: для каждого результата получен ответ. Посмотрим на результаты.

In [43]:
# выведем последние пять строк датасета df
df.tail()

,a,b,test
99995,46,50,False
99996,44,57,False
99997,46,64,False
99998,41,54,False
99999,49,52,False


In [44]:
# посчитаем процент результатов True с помощью среднего
df['test'].mean()

0.31956

Если реальная разница составляет 1%, то статистический тест обнаруживет этот эффект только примерно в 30% случаев. Мы установили **Power (Desired minimum True Positive Rate)** на уровне 80%, следовательно, величина в 30% не может быть минимальным эффектом. Минимальный эффект у этого теста больше.

Посмотрим, какая мощность будет получена, если разница конверсий составит 1.5%.

In [45]:
# запускаем эксперимент 100_000 раз
# для каждого запуска получим пару конверсий для тестовой и контрольной группы
n = 100_000
result = []

for _ in tqdm(range(n)):
    a = np.random.binomial(1_484, 0.03)
    b = np.random.binomial(1_484, 0.045)
    result.append((a,b))

100%|██████████| 100000/100000 [00:00<00:00, 352443.86it/s]


In [46]:
# сохраним результаты в таблицу
df = pd.DataFrame(result, columns = ['a', 'b'])

In [47]:
# выведем первые пять строк таблицы result
df.head()

,a,b
0,41,70
1,42,57
2,46,63
3,39,47
4,52,58


Посчитаем результат каждого виртуального теста, используя функцию **test**:

* True - решение положительное: считаем, что разница в конверсиях между двумя группами есть;

* False - решение отрицательное: считаем, что разницы в конверсиях между двумя группами нет.

In [48]:
# результат каждого виртуального теста True / False
# сохраним в отдельный столбец датасета df
tqdm.pandas()

df['test'] = df.progress_apply(
    lambda row: test(row['a'], row['b'], 1484, 1484), axis=1)

100%|██████████| 100000/100000 [00:19<00:00, 5198.05it/s]


Тесты рассчитались: для каждого результата получен ответ. Посмотрим на результаты.

In [49]:
# выведем последние пять строк датасета df
df.tail()

,a,b,test
99995,41,73,True
99996,44,73,True
99997,48,56,False
99998,46,60,False
99999,48,77,True


In [50]:
# посчитаем процент результатов True с помощью среднего
df['test'].mean()

0.57985

Мощность выросла с примерно 30% до примерно 58%. То есть мы стали ближе к реальному эффекту. Если последовательно повышать разницу в конверсиях мы будем все ближе приближаться к мощности в 80%, которая достигается при разнице в конверсиях в 2% (0.03 в группе 'a' против 0.05 в группе 'b').

**Таким образом** можно протестировать и **Minimal Detectable Effect**, который призван выявить тест. Для этого проводим несколько запусков, пробуя разные варианты. Как вариант можно провести серию запусков с определенным шагом и посмотреть, как меняется **Power (Desired minimum True Positive Rate)** в зависимости от выбранного размера эффекта, и считать **реальным эффектом** теста, тот эффект, при котором будет достигаться **Power (Desired minimum True Positive Rate)**.

## <font color='green'>**3 Оценка результатов A/B-теста с помощью метода Монте-Карло и онлайн-калькулятора Эвана Миллера на этапе планирования эксперимента**</font>

---

Во втором разделе оценка результатов A/B-теста на этапе планирования эксперимента была реализована с помощью **онлайн-калькулятора Глеба Михайлова**.

Проведем аналогичную проверку, но с использованием **Sample Size Calculator Эвана Миллера**.

Предварительно в **Sample Size Calculator Эвана Миллера** рассчитали необходимый размер группы **1245** при следующих параметрах:

* Baseline Conversion Rate 0.03

* Minimum Detectable Effect 0.02

* Test to Control Group Ratio 1.0

* Power (Desired minimum True Positive Rate) 0.8

* Significance (Desired maximum False Positive Rate) 0.05

Напомним, что с поомщью **онлайн-калькулятора Глеба Михайлова** необходимый размер выборки был определен на уровне **1484**.

Ссылка на **Sample Size Calculator Эвана Миллера**:

https://www.evanmiller.org/ab-testing/sample-size.html

Для размера выборки, определенной с помощью **Sample Size Calculator Эвана Миллера** количество экспериментов установим равным 100_000, - для повышения точности результатов применения **метода Монте-Карло**.

In [51]:
# вычислим средний размер конверсии в контрольной и тестовой группах

np.random.seed(7)

a = np.random.binomial(1, 0.03, size = 1_245).mean()
b = np.random.binomial(1, 0.05, size = 1_245).mean()

print(a, b)

0.02650602409638554 0.04819277108433735


Предварительная оценка с помощью виртуального эксперимента показывает, что статистический тест может уловить разницу в конверсиях в 0.02 при размере выборки в **1245**. При этом, планируя эксперимент, мы заранее знаем, что конверсии в двух группах различаются.

В аргументах функции **np.random.binomial** уберем размер выборки size, а первый аргумент заменим на размер выборки, вероятность успеха сохраним. В результате функция выведет количество **'успехов'** при заданной вероятности.

In [52]:
# получим число конверсий
# в контрольной и тестовой группах

np.random.seed(7)

a = np.random.binomial(1_245, 0.03)
b = np.random.binomial(1_245, 0.05)

print(a, b)

32 64


Однако, без фиксации **np.random.seed()** в один из экспериментальных запусков было получено число конверсий для группы **а** 55, для группы **b** 61. Результат, полученный с помощью **онлайн-калькулятора Глеба Михайлова**: принимаем нулевую гипотезу, разницы в конверсии между двумя группами нет. Хотя, планируя эксперимент, мы точно знаем, что разница в конверсии в двух группах есть.

Это **False Negative** - мы пропустили эффект, когда в действительности он есть. Вероятность такой ошибки заложена в калькулятор и составляет 5% (**уровень значимости**).

Воспользуемся функцией **test** из предыдущего раздела.

In [53]:
# функция, позволяющая смоделировать многократный запуск эксперимента
# и рассчитать мощность при заданных параметрах;
# conv_a и conv_b - конверсии в группах a и b

def test(conv_a, conv_b, size_a, size_b, significance=0.05):
    _, p_value = proportions_ztest(
        [conv_a, conv_b],
        [size_a, size_b],
        alternative='two-sided')
    return p_value < significance

При конверсиях в группах a и b **онлайн-калькулятор Глеба Михайлова** дал отрицательный ответ: не отвергаем нулевую гипотезу о равенстве конверсий в группах, хотя в реальности конверси ив группах различаются.

Проверим, какй результат будет получен в результате применения **proportion_ztest** на выборке, размер которой определен с помощью **Sample Size Calculator Эвана Миллера**.

In [54]:
# применим функцию test к размерам конверсий
# в группах a и b в 49 и 52
test(55, 61, 1245, 1245)

False

**False** означает, что ответ отрицательный, различий в конверсиях двух групп нет (что не соответствует реальному положению дел).

Протестируем функцию test на значениях конверсии, которые точно должны показать наличие развитий: 32 и 64 соответственно для группы **a** и группы **b**.

In [55]:
# применим функцию test к размерам конверсий
# в группах a и b в 44 и 78
test(32, 64, 1245, 1245)

True

**True** означает, что результат положительный: отвергаем нулевую гипотезу о равенстве значений конверсий, конверсии двух групп различаются (что соответствует реальности).

На следующих шагах проведем множество виртуальных экспериментов и проверим, получаем ли мы желаемые метрики статистического теста, или нет.

### <font color='green'>**3.1 Оценка True Positive Rate (Sensitivity)**</font>

---

Сгенерируем 1000 экспериментов для набора конверсий 32 (для группы '**a**') и 64 (для группы '**b**') и для каждого эксперимента посчитаем ответ True или False. После чего посчитаем число ответов  True и поделим на общее количество ответов.

In [56]:
# запускаем эксперимент 100_000 раз
# для каждого запуска получим пару конверсий для тестовой и контрольной группы
n = 100_000
result = []

for _ in tqdm(range(n)):
    a = np.random.binomial(1_245, 0.03)
    b = np.random.binomial(1_245, 0.05)
    result.append((a,b))

100%|██████████| 100000/100000 [00:00<00:00, 354020.51it/s]


In [57]:
# сохраним результаты в таблицу
df = pd.DataFrame(result, columns = ['a', 'b'])

In [58]:
# выведем первые пять строк таблицы result
df.head()

,a,b
0,33,63
1,45,58
2,38,61
3,55,61
4,41,61


In [59]:
# результат каждого виртуального теста True / False
# сохраним в отдельный столбец датасета df
df['test'] = df.progress_apply(lambda row: test(row['a'], row['b'], 1245, 1245), axis=1)

100%|██████████| 100000/100000 [00:17<00:00, 5615.72it/s]


In [60]:
# выведем последние пять строк датасета df
df.tail()

,a,b,test
99995,31,63,True
99996,43,58,False
99997,33,74,True
99998,27,58,True
99999,42,63,True


Сходу можно увидеть, что среди результатов есть False, хотя мы знаем заранее, что таких результатов быть не должно - конверсии в двух группах точно различаются.

Поскольку при расчете начальных параметров мы заложили желаемую мощность теста (**True Positive Rate**) в 80%, результат **False** мы должны увидеть в 20% случаев, а **True** должно быть в 80% случаев.

In [61]:
# посчитаем процент результатов True с помощью среднего
df['test'].mean()

0.72457

Желаемая мощность **Power (Desired minimum TPR)** должна быть не менее 80%, поэтому результат ниже 73% нас не устраивает.

Добавим, что у **метода Монте-Карло** есть ограничение по точности, которое определяется количеством проводимых виртуальных экспериментов. Если нам требуется более точная оценка для **TPR** теста, то нам следует повысить количество наблюдений. Например, можно провести не 1_000, а 10_000 виртуальных экспериментов.

Чем меньше MDE теста, тем больше повторений при проведени вирутальных экспериментов **методом Монте-Карло** требуется.

**В результате** применения случайных чисел, виртуальных экспериментов, метода Монте-Карло **не** удалось подтвердить, что тест, который был запланирован, действительно обладает таким **Power (Desired minimum TPR)**, который в него был заложен. Была проведена серия из 100_000 экспериментов, причем заранее было известно, что разница есть. Но тест обнаружил эту разницу только примрено в 73% случаев. Это не соответствует тому, что было заложено в тест при планировании эксперимента.

### <font color='green'>**3.2 Оценка False Positive Rate (Significance)**</font>

---

Для оценки уровня значимости (**False Positive Rate**) в цикле, в котором формируются случайные величины конверсий в тестовой и контрольной группах, укажем равные конверсии. Теперь на этапе планирования эксперимента предполагаем, что разницы в уровне конверсий между двумя группами **нет**.

In [62]:
# запускаем эксперимент 100_000 раз
# для каждого запуска получим пару конверсий для тестовой и контрольной группы
n = 100_000
result = []

for _ in tqdm(range(n)):
    a = np.random.binomial(1_245, 0.03)
    b = np.random.binomial(1_245, 0.03)
    result.append((a,b))

100%|██████████| 100000/100000 [00:00<00:00, 379799.81it/s]


In [63]:
# сохраним результаты в таблицу
df = pd.DataFrame(result, columns = ['a', 'b'])

In [64]:
# выведем первые пять строк таблицы result
df.head()

,a,b
0,48,34
1,37,28
2,32,35
3,31,33
4,36,35


Каждая строка датасета df - результат отдельного виртуального эксперимента.

Проведено 100_000 виртуальных экспериментов, в которых мы заранее знаем, что никакой разницы между конверсиями в тестовой и контрольной группах нет.

Посчитаем результат каждого виртуального теста, используя функцию **test**:

* True - решение положительное: считаем, что разница в конверсиях между двумя группами есть;

* False - решение отрицательное: считаем, что разницы в конверсиях между двумя группами нет.

In [65]:
# результат каждого виртуального теста True / False
# сохраним в отдельный столбец датасета df
tqdm.pandas()

df['test'] = df.progress_apply(
    lambda row: test(row['a'], row['b'], 1245, 1245), axis=1)

100%|██████████| 100000/100000 [00:19<00:00, 5240.34it/s]


Тесты рассчитались: для каждого результата получен ответ. Посмотрим на результаты.

In [66]:
# выведем последние lдесять строк датасета df
df.tail(10)

,a,b,test
99990,31,28,False
99991,42,36,False
99992,38,32,False
99993,28,36,False
99994,46,36,False
99995,28,34,False
99996,31,27,False
99997,30,37,False
99998,39,35,False
99999,50,36,False


В качестве результата ожидаются только **False**, однако же среди результатов виртуальных экспериментов встречаются и **True**, которых здесь быть не должно, т.к. мы точно знаем, что разницы в конверсиях между двумя группами нет.

Однако, статистический тест не идеален, и в 5% случаев, когда разницы нет, тест допускает ложное срабатывание, - он дает ответ, что разница есть.

In [67]:
# посчитаем процент результатов True с помощью среднего
df['test'].mean()

0.05005

По итогам проведения серии из 100 тысяч экспериментов и усреднения полученных результатов, оценка уровня значимости (**False Positive Rate**) оказалась равна 5%, как и было заложено при планировании теста. Размера выборки, рассчитанной с помощью **Sample Size Calculator Эвана Миллера** достаточно, чтобы ошибка 1-го рода была равна заявленным 5%.

### <font color='green'>**3.3 Оценка Minimal Detectable Effect (MDE)**</font>

---

Вернемся к ситуации, когда разница в конверсии между группами есть. И реальная разница соответствует **Minimal Detectable Effect**.

Запустим проверку и посмотрим, в каком количестве случаев тест обнаруживает разницу на выборке в **1245** пользователей, расчитанную с помощью **Sample Size Calculator Эвана Миллера**.

In [68]:
# запускаем эксперимент 100_000 раз
# для каждого запуска получим пару конверсий для тестовой и контрольной группы
n = 100_000
result = []

for _ in tqdm(range(n)):
    a = np.random.binomial(1_245, 0.03)
    b = np.random.binomial(1_245, 0.05)
    result.append((a,b))

100%|██████████| 100000/100000 [00:00<00:00, 198596.57it/s]


In [69]:
# сохраним результаты в таблицу
df = pd.DataFrame(result, columns = ['a', 'b'])

In [70]:
# выведем первые пять строк таблицы result
df.head()

,a,b
0,49,52
1,47,69
2,31,75
3,42,48
4,42,73


Посчитаем результат каждого виртуального теста, используя функцию **test**:

* True - решение положительное: считаем, что разница в конверсиях между двумя группами есть;

* False - решение отрицательное: считаем, что разницы в конверсиях между двумя группами нет.

In [71]:
# результат каждого виртуального теста True / False
# сохраним в отдельный столбец датасета df
tqdm.pandas()

df['test'] = df.progress_apply(
    lambda row: test(row['a'], row['b'], 1245, 1245), axis=1)

100%|██████████| 100000/100000 [00:18<00:00, 5493.55it/s]


Тесты рассчитались: для каждого результата получен ответ. Посмотрим на результаты.

In [72]:
# посчитаем процент результатов True с помощью среднего
df['test'].mean()

0.72738

Видим, что тест обнаруживает **Minimal Detectable Effect (MDE)** в чуть более чем 72% случаев. Выборки в размере **1245**, рассчитанной с помощью **Sample Size Calculator Эвана Миллера** недостаточно, чтобы обнаружить MDE при запланированных параметрах теста.

Проверим, что произойдет, если MDE понизить с 0.05 до 0.04. То есть предположим, что конверсия в контрольной группе 0.03, а в тестовой не 0.05, а 0.04.

In [73]:
# запускаем эксперимент 100_000 раз
# для каждого запуска получим пару конверсий для тестовой и контрольной группы
n = 100_000
result = []

for _ in tqdm(range(n)):
    a = np.random.binomial(1_245, 0.03)
    b = np.random.binomial(1_245, 0.04)
    result.append((a,b))

100%|██████████| 100000/100000 [00:00<00:00, 373161.71it/s]


In [74]:
# сохраним результаты в таблицу
df = pd.DataFrame(result, columns = ['a', 'b'])

Посчитаем результат каждого виртуального теста, используя функцию **test**:

* True - решение положительное: считаем, что разница в конверсиях между двумя группами есть;

* False - решение отрицательное: считаем, что разницы в конверсиях между двумя группами нет.

In [75]:
# результат каждого виртуального теста True / False
# сохраним в отдельный столбец датасета df
tqdm.pandas()

df['test'] = df.progress_apply(
    lambda row: test(row['a'], row['b'], 1245, 1245), axis=1)

100%|██████████| 100000/100000 [00:18<00:00, 5524.04it/s]


Тесты рассчитались: для каждого результата получен ответ. Посмотрим на результаты.

In [76]:
# посчитаем процент результатов True с помощью среднего
df['test'].mean()

0.27554

Если реальная разница составляет 1%, то статистический тест обнаруживет этот эффект только примерно в 27% случаев. Мы установили **Power (Desired minimum True Positive Rate)** на уровне 80%, следовательно, величина в 30% не может быть минимальным эффектом. Минимальный эффект у этого теста больше.

Посмотрим, какая мощность будет получена, если разница конверсий составит 1.5%.

In [77]:
# запускаем эксперимент 100_000 раз
# для каждого запуска получим пару конверсий для тестовой и контрольной группы
n = 100_000
result = []

for _ in tqdm(range(n)):
    a = np.random.binomial(1_245, 0.03)
    b = np.random.binomial(1_245, 0.045)
    result.append((a,b))

100%|██████████| 100000/100000 [00:00<00:00, 207826.47it/s]


In [78]:
# сохраним результаты в таблицу
df = pd.DataFrame(result, columns = ['a', 'b'])

Посчитаем результат каждого виртуального теста, используя функцию **test**:

* True - решение положительное: считаем, что разница в конверсиях между двумя группами есть;

* False - решение отрицательное: считаем, что разницы в конверсиях между двумя группами нет.

In [79]:
# результат каждого виртуального теста True / False
# сохраним в отдельный столбец датасета df
tqdm.pandas()

df['test'] = df.progress_apply(
    lambda row: test(row['a'], row['b'], 1245, 1245), axis=1)

100%|██████████| 100000/100000 [00:18<00:00, 5294.76it/s]


Тесты рассчитались: для каждого результата получен ответ. Посмотрим на результаты.

In [80]:
# посчитаем процент результатов True с помощью среднего
df['test'].mean()

0.50571

Мощность выросла с примерно 27% до примерно 50%. То есть мы стали ближе к реальному эффекту. Если последовательно повышать разницу в конверсиях мы будем все ближе приближаться к мощности в 80%. Однако, как было показано выше, при размере выборки в **1245** обнаружить разницу в конверсии в 80% случаев тест не может. Чтобы добиться планируемого значения мощности в 80% необходимо либо повысить величину MDE, либо увеличить размер выборки.

Таким образом, **онлайн-калькулятор Глеба Михайлова** предоставил более точную оценку по требуемому размеру выборки на этапе планирования A/B-теста, чем **Sample Size Calculator Эвана Миллера**.